# One-hot encoding features

In [1]:
import pandas as pd


- Import from `fetched_data`

In [2]:
techniques_df               = pd.read_csv('fetched_data/techniques_df.csv')
techniques_mitigations_df   = pd.read_csv('fetched_data/techniques_mitigations_df.csv')
groups_df                   = pd.read_csv('fetched_data/groups_df.csv')
groups_techniques_df        = pd.read_csv('fetched_data/groups_techniques_df.csv')
groups_software_df          = pd.read_csv('fetched_data/groups_software_df.csv')
tactics_df                  = pd.read_csv('fetched_data/tactics_df.csv')
software_df                 = pd.read_csv('fetched_data/software_df.csv')
software_groups_df          = pd.read_csv('fetched_data/software_groups_df.csv')


# 1- Group-Technique matrix (user-item)
- each line of the matrix represents a Group-Technique relationship

In [3]:
g_t_df = groups_techniques_df [['source ID', 'target ID',]]
g_t_df.columns = ['group ID', 'technique ID']
g_t_df.head()

,group ID,technique ID
0,G0099,T1105
1,G0099,T1204.002
2,G0099,T1036.004
3,G0099,T1571
4,G0099,T1027


---
# 2- Group: constructing features

- Turn software into one-hot feature for groups
- Two dataframes to merge based on group ID are
    1. group id list
    2. group - software one-hot encoded

In [4]:
groups_software_df.head()

,source ID,source name,source type,mapping type,target ID,target name,target type,mapping description
0,G0099,APT-C-36,group,uses,S0434,Imminent Monitor,software,(Citation: QiAnXin APT-C-36 Feb2019)
1,G0006,APT1,group,uses,S0017,BISCUIT,software,(Citation: Mandiant APT1)
2,G0006,APT1,group,uses,S0025,CALENDAR,software,(Citation: Mandiant APT1)
3,G0006,APT1,group,uses,S0119,Cachedump,software,(Citation: Mandiant APT1)
4,G0006,APT1,group,uses,S0026,GLOOXMAIL,software,(Citation: Mandiant APT1)


In [5]:
print (groups_software_df['source ID'].nunique())
print (groups_software_df['target ID'].nunique())

124
463


🔍 There are 127 unique Group IDs in `groups_software_df`. Total number of Groups: 136

🔍 There are 463 unique Software IDs in `groups_software_df`. Total number of Software: 635

In [6]:
groups_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 136 entries, 0 to 135
Data columns (total 11 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   ID                           136 non-null    object 
 1   name                         136 non-null    object 
 2   description                  136 non-null    object 
 3   url                          136 non-null    object 
 4   created                      136 non-null    object 
 5   last modified                136 non-null    object 
 6   version                      136 non-null    float64
 7   contributors                 73 non-null     object 
 8   associated groups            78 non-null     object 
 9   associated groups citations  77 non-null     object 
 10  relationship citations       136 non-null    object 
dtypes: float64(1), object(10)
memory usage: 11.8+ KB


- 🔍 There are 136 Groups, in which 124 Groups use one ore more software 👉 Features for the other groups (that doesn't use any software) will be filled with NaN values (0)

In [7]:
groups_software_df.head()

,source ID,source name,source type,mapping type,target ID,target name,target type,mapping description
0,G0099,APT-C-36,group,uses,S0434,Imminent Monitor,software,(Citation: QiAnXin APT-C-36 Feb2019)
1,G0006,APT1,group,uses,S0017,BISCUIT,software,(Citation: Mandiant APT1)
2,G0006,APT1,group,uses,S0025,CALENDAR,software,(Citation: Mandiant APT1)
3,G0006,APT1,group,uses,S0119,Cachedump,software,(Citation: Mandiant APT1)
4,G0006,APT1,group,uses,S0026,GLOOXMAIL,software,(Citation: Mandiant APT1)


## 2-1 Group ID list

In [8]:
g_id_df = groups_df[['ID']]
g_id_df.columns = ['group ID']

In [9]:
g_id_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 136 entries, 0 to 135
Data columns (total 1 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   group ID  136 non-null    object
dtypes: object(1)
memory usage: 1.2+ KB


## 2-2 Group - software one-hot encoded

In [10]:
g_s_df = groups_software_df[['source ID', 'target ID']]
g_s_df.columns = ['group ID', 'software ID']


# extract the id
id_g_s_df = g_s_df[['group ID']]

# one-hot encode the features
g_s_ohf = pd.get_dummies(g_s_df['software ID'], dtype= float)
# g_s_ohf: group-software one-hot feature

g_s_ohf = pd.concat(
    [
        id_g_s_df, 
        g_s_ohf
    ],
    axis = 1
)

g_s_ohf = g_s_ohf.groupby('group ID').max().reset_index()

g_s_ohf.head()

,group ID,S0002,S0003,S0004,S0005,S0006,S0008,S0009,S0010,S0012,...,S1035,S1037,S1039,S1046,S1047,S1051,S1058,S1059,S1060,S1072
0,G0001,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,G0002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,G0003,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,G0004,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,G0005,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
print (g_s_ohf.shape)

(124, 464)


## 2-3 Merge the dataframes

- After merging, the Techniques that are not used by a Group will be `NaN` -> Replace `NaN` with 0's

In [12]:
g_df = pd.merge (g_id_df, g_s_ohf, on = 'group ID', how = 'left')

g_df.fillna(0, inplace= True)

In [13]:
g_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 136 entries, 0 to 135
Columns: 464 entries, group ID to S1072
dtypes: float64(463), object(1)
memory usage: 493.1+ KB


# 3- Technique Features
- features chosen for techniques are
    1. Platforms
    2. Mitigations

- three dataframes to merge based on technique ID are:
    1. technique id list
    2. technique - platforms one-hot encoded
    3. technique - mitigations one-hot encoded

In [14]:
techniques_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 607 entries, 0 to 606
Data columns (total 21 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   ID                      607 non-null    object 
 1   name                    607 non-null    object 
 2   description             607 non-null    object 
 3   url                     607 non-null    object 
 4   created                 607 non-null    object 
 5   last modified           607 non-null    object 
 6   version                 607 non-null    float64
 7   tactics                 607 non-null    object 
 8   detection               579 non-null    object 
 9   platforms               607 non-null    object 
 10  data sources            568 non-null    object 
 11  is sub-technique        607 non-null    bool   
 12  sub-technique of        411 non-null    object 
 13  defenses bypassed       103 non-null    object 
 14  contributors            349 non-null    ob

## 3-1 Technique id list

In [15]:
t_id_df = techniques_df[['ID']]
t_id_df.columns = ['technique ID']
t_id_df

,technique ID
0,T1548
1,T1548.002
2,T1548.004
3,T1548.001
4,T1548.003
...,...
602,T1102.002
603,T1102.001
604,T1102.003
605,T1047


## 3-2 Technique - platforms one-hot encoded

In [16]:
t_p_df = techniques_df[['ID', 'platforms']]
# t-p-df: technique-platform 
id_t_p_df = t_p_df[['ID']]
t_p_ohf_df = t_p_df['platforms'].str.get_dummies (sep = ', ')
# t_p_ohf_df: technique-platform one-hot-feature

t_p_ohf_df = pd.concat (
    [id_t_p_df, t_p_ohf_df],
    axis = 1
)
t_p_ohf_df.rename (columns= {'ID': 'technique ID'}, inplace= True)


In [17]:
t_p_ohf_df.head()

,technique ID,Azure AD,Containers,Google Workspace,IaaS,Linux,Network,Office 365,PRE,SaaS,Windows,macOS
0,T1548,0,0,0,0,1,0,0,0,0,1,1
1,T1548.002,0,0,0,0,0,0,0,0,0,1,0
2,T1548.004,0,0,0,0,0,0,0,0,0,0,1
3,T1548.001,0,0,0,0,1,0,0,0,0,0,1
4,T1548.003,0,0,0,0,1,0,0,0,0,0,1


In [18]:
t_p_ohf_df.shape

(607, 12)

## 3-3 Technique - mitigation one-hot encoded

In [19]:
# get the dataframe
t_m_df = techniques_mitigations_df[['source ID', 'target ID']]
t_m_df.columns = ['mitigation ID', 'technique ID']

# extract the id
id_t_m_df = t_m_df[['technique ID']]

# one-hot encode the feature
t_m_ohf_df = pd.get_dummies(t_m_df['mitigation ID'], dtype= float)

# concat the id with the one-hot encoded
t_m_ohf_df = pd.concat (
    [id_t_m_df, t_m_ohf_df],
    axis= 1
)

t_m_ohf_df = t_m_ohf_df.groupby('technique ID').max().reset_index()

`one_hot_features.groupby('technique ID').max().reset_index()`: This line groups the `one_hot_features` DataFrame by the 'technique ID' column and applies the `max()` function to each group. Since we are working with one-hot encoded values, the `max()` function will retain 1 for each genre if it exists in any row for a specific technique ID, and 0 if the genre is not present for that technique ID. The `reset_index()` function is used to reset the index of the resulting DataFrame, ensuring that the 'technique ID' column becomes a regular column.

In [20]:
t_m_ohf_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 514 entries, 0 to 513
Data columns (total 44 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   technique ID  514 non-null    object 
 1   M1013         514 non-null    float64
 2   M1015         514 non-null    float64
 3   M1016         514 non-null    float64
 4   M1017         514 non-null    float64
 5   M1018         514 non-null    float64
 6   M1019         514 non-null    float64
 7   M1020         514 non-null    float64
 8   M1021         514 non-null    float64
 9   M1022         514 non-null    float64
 10  M1024         514 non-null    float64
 11  M1025         514 non-null    float64
 12  M1026         514 non-null    float64
 13  M1027         514 non-null    float64
 14  M1028         514 non-null    float64
 15  M1029         514 non-null    float64
 16  M1030         514 non-null    float64
 17  M1031         514 non-null    float64
 18  M1032         514 non-null    

## 3-4 Merge the dataframes

In [21]:
t_df = pd.merge (t_id_df, t_p_ohf_df, on='technique ID', how= 'left')

In [22]:
t_df = pd.merge (t_df, t_m_ohf_df, on = 'technique ID', how = 'left' )

In [23]:
t_df.fillna(0, inplace= True)

In [24]:
t_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 607 entries, 0 to 606
Data columns (total 55 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   technique ID      607 non-null    object 
 1   Azure AD          607 non-null    int64  
 2   Containers        607 non-null    int64  
 3   Google Workspace  607 non-null    int64  
 4   IaaS              607 non-null    int64  
 5   Linux             607 non-null    int64  
 6   Network           607 non-null    int64  
 7   Office 365        607 non-null    int64  
 8   PRE               607 non-null    int64  
 9   SaaS              607 non-null    int64  
 10  Windows           607 non-null    int64  
 11  macOS             607 non-null    int64  
 12  M1013             607 non-null    float64
 13  M1015             607 non-null    float64
 14  M1016             607 non-null    float64
 15  M1017             607 non-null    float64
 16  M1018             607 non-null    float64
 1

# 4- Target matrix
- There are three columns in `target` matrix:
    1. Group ID
    2. Technique ID
    3. `target` - binary value: 
        - `1` if Group uses the Technique, or 
        - `0` if Group does NOT use the Technique

In [25]:
print (g_t_df.shape)

print (g_id_df.shape)
print (g_df.shape)

print (t_df.shape)
print (t_id_df.shape)

(3052, 2)
(136, 1)
(136, 464)
(607, 55)
(607, 1)


## 4-0 Group ID - Technique ID Cartesian product (`G_T_id_df`)
- Cartesian product of Group and Technique will have 136 $\times$ 607 = 82552 instances

In [26]:
G_T_id_df = pd.merge (g_id_df, t_id_df, how = 'cross')

In [27]:
G_T_id_df.shape

(82552, 2)

# 4-1 Positive instances
- The istances where a Group uses a Technique is stored in `g_t_target_df`

In [28]:
g_t_target_df = g_t_df.copy()
g_t_target_df['target'] = 1

In [29]:
g_t_target_df

,group ID,technique ID,target
0,G0099,T1105,1
1,G0099,T1204.002,1
2,G0099,T1036.004,1
3,G0099,T1571,1
4,G0099,T1027,1
...,...,...,...
3047,G0045,T1588.002,1
3048,G0045,T1199,1
3049,G0045,T1078,1
3050,G0045,T1059.003,1


# 4-3 Merging `G_T_id_df` with `g_t_target_df`

In [30]:
G_T_df = pd.merge (G_T_id_df, g_t_target_df, on = ['group ID', 'technique ID'], how = 'left')

In [31]:
G_T_id_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82552 entries, 0 to 82551
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   group ID      82552 non-null  object
 1   technique ID  82552 non-null  object
dtypes: object(2)
memory usage: 1.3+ MB


- Fill NaN values in `target` with `0`'s

In [32]:
G_T_df['target'].fillna (0, inplace= True)

In [33]:
G_T_df['target'].value_counts()

target
0.0    79500
1.0     3052
Name: count, dtype: int64

---
# 5 Export

In [34]:
dfs = {
    "g_df": g_df,
    "t_df": t_df,
    "G_T_df": G_T_df
}

In [35]:
for key in dfs.keys():
    dfs[key].to_csv (f"preprocessed_data/{key}.csv", index = False)